In [ ]:
from pymongo import MongoClient

MONGODB_URI = "mongodb://localhost:27017/"

client = MongoClient(MONGODB_URI)
db = client["surreal_deal"]

person = db["person"]
product = db["product"]
order = db["order"]
artist = db["artist"]
review = db["review"]

In [ ]:
%%timeit
list(review.aggregate([
	{
		"$lookup": {
			"from": "artist",
			"localField": "artist",
			"foreignField": "_id",
			"pipeline": [
				{ "$project": { "_id": 0, "name": 1, "email": 1, "phone": 1 } }
			],
			"as": "artist",
		}

	},
	{
		"$lookup": {
			"from": "person",
			"localField": "person",
			"foreignField": "_id",
			"pipeline": [
				{ "$project": { "_id": 0, "name": 1, "email": 1, "phone": 1 } }
			],
			"as": "person",
		}

	},
	{
		"$lookup": {
			"from": "product",
			"localField": "product",
			"foreignField": "_id",
			"pipeline": [
				{ "$project": { "_id": 0, "name": 1, "category": 1, "price": 1 } }
			],
			"as": "product",
		}
	}
]))

In [ ]:
%%timeit
list(order.aggregate([
	{
		"$lookup": {
			"from": "person",
			"localField": "person",
			"foreignField": "_id",
			"pipeline": [
				{ "$project": { "_id": 0, "name": 1, "email": 1, "phone": 1 } }
			],
			"as": "person",
		}
	},
	{
		"$lookup": {
			"from": "product",
			"localField": "product",
			"foreignField": "_id",
			"pipeline": [
				{
					"$lookup": {
						"from": "artist",
						"localField": "artist",
						"foreignField": "_id",
						"pipeline": [
							{ "$project": { "_id": 0, "name": 1, "email": 1, "phone": 1 } }
						],
						"as": "artist",
					}
				},
				{ "$project": { "_id": 0, "category": 1, "description": 1, "image_url": 1, "artist": 1 } }
			],
			"as": "product",
		}
	},
	{ "$project": { "_id": 0, "price": 1, "order_date": 1, "product_name": 1, "artist": 1, "person": 1, "product": 1 } }
]))

In [ ]:
%%timeit
list(person.find(
	{ "address.country": "England" },
	{ "_id": 0, "name": 1, "email": 1 }
))

In [54]:
%%timeit
list(order.aggregate([
	{
		"$lookup": {
			"from": "product",
			"localField": "product",
			"foreignField": "_id",
			"pipeline": [
				{
					"$lookup": {
						"from": "artist",
						"localField": "artist",
						"foreignField": "_id",
						"pipeline": [
							{ "$project": { "_id": 0, "address.country": 1 } }
						],
						"as": "artist",
					}
				},
				{ "$project": { "_id": 0, "artist": 1 } }
			],
			"as": "product",
		}
	},
	{
		"$match": {
			"$or": [{ "order_status": "null" }, { "order_status": { "$ne": "pending" } }],
			"$expr": { "$lte": [{ "$month": "$order_date" }, 3] },
			"product.artist.address.country": "England",
		}
	},
	{ "$count": "count" }
]))

115 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [56]:
review.delete_one({ "_id": "716f6bed-60b4-4e6d-964f-3135040d69df" }) # <- TODO have it select a random id to delete so it can be run in a loop

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [57]:
review.delete_many({ "product": { "$in": product.distinct("_id", { "category": "charcoal" }) } })


DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [58]:
person.update_one(
	{ "_id": "41e60664-e96c-465d-b92a-eee2f143c05d" }, # <- TODO have it select a random id to delete so it can be run in a loop
	{
		"$set": {
			"address": {
				'address_line_1': '497 Ballycander',
				'address_line_2': "null",
				'city': 'Bromyard',
				'country': 'Wales',
				'post_code': 'ZX8N 4VJ',
				'coordinates': [68.772592, -35.491877]
			}
		}
	}
)


UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [53]:
product.update_many(
	{ "price": { "$lt": 1000 } },
	{ "$set": { "discount": 0.2 } }
)


UpdateResult({'n': 16, 'nModified': 16, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [ ]:
# Transaction - order from a new customer

In [61]:
# Transaction - New Artist creates their first product

# Step 1: Define the callback that specifies the sequence of operations to perform inside the transactions.
def callback(session):
    # link collections to the session
    artist_collection = session.client.surreal_deal.artist
    person_collection = session.client.surreal_deal.person

	# insert into the artist table
    artist_collection.insert_one({
        '_id': 'b8683656-6fe3-47ae-830c-41451c3ce953',
        'first_name': 'Anderson',
        'last_name': 'West',
        'name': 'Anderson West',
        'company_name': 'Atkins(ws) (ATK)',
        'email': 'six1933@gmail.com',
        'phone': '056 5881 1126',
        'address': {
            'address_line_1': '639 Connaugh',
            'address_line_2': "null",
            'city': 'Ripon',
            'country': 'Scotland',
            'post_code': 'CG3U 4TH',
            'coordinates': [4.273648, -112.907273]
		}
	},
    session=session)

	# insert into the product table
	person_collection.insert_one({
        '_id': 'd8f59414-4484-4634-a866-8b607fd2311e',
        'name': 'managed edt allocated pda',
        'description': 'counseling dildo greek pan works interest xhtml wrong dennis available cl specific next tower webcam peace magic',
        'category': 'watercolor',
        'price': 15735.96,
        'currency': '£',
        'discount': "null",
        'quantity': 1,
        'image_url': 'https://source.unsplash.com/1920x1080?',
        "artist": 'b8683656-6fe3-47ae-830c-41451c3ce953',
        "creation_history": {
            "quantity": 1,
            "created_at": "new Date()"
		}
	},
    session=session)

# Step 2: Start a client session.
with client.start_session() as session:
    # Step 3: Use with_transaction to start a transaction, execute the callback, and commit (or abort on error).
    session.with_transaction(
        callback,
        read_concern=ReadConcern("snapshot"),
        write_concern=WriteConcern("majority", wtimeout=1000),
        read_preference=ReadPreference.PRIMARY,
    )

TabError: inconsistent use of tabs and spaces in indentation (45388290.py, line 30)